# Decision trees




In this section we cover the decision tree methods. We divided it into three parts:
- univariate methods,
- multivariate methods,
- quality metrics.

The customer segmentation shown in the lectures looks as following:


| Location|Category   | Gender | Product review  | Decision |
|---------|-----------|--------|-----------------|--------|
| Berlin | Furniture  | Male   | Checked Reviews | Bought |
| London | Furniture  | Male   | Checked Reviews | Bought |
| Berlin | Furniture  | Female | Checked Reviews | Did not bought |
| Berlin | Textile    | Female | Checked Reviews | Bought |
| London | Electronics| Male   | Checked Reviews | Did not bough |
| London | Textile    | Female | Checked Reviews | Did not bough |
| Paris  | Textile    | Male   | Did not checked | Bought |
| Berlin | Electronics| Male   | Checked Reviews | Bought |
| Paris  | Electronics| Male   | Did not checked | Bought |
| London | Electronics| Female | Checked Reviews | Bought |
| Paris  | Furniture  | Female | Did not checked | Bought |
| Berlin | Textile    | Female | Did not checked | Bought |
| London | Electronics| Female | Did not checked | Did not bough |
| London | Furniture  | Female | Checked Reviews | Did not bough |
| London | Textile    | Female | Did not checked | Did not bough |

In [1]:
import numpy

labels = [1,1,-1,1,-1,-1,1,1,1,1,1,1,-1,-1,-1]
X = [[1,1,2,2],[2,1,2,2],[1,1,1,2],[1,2,1,2],[2,3,2,2],
                [2,2,1,2],[3,2,2,1],[1,3,2,2],[3,3,2,1],[2,3,1,2],
                [3,1,1,1],[1,2,1,1],[2,3,1,1],[2,1,1,2],[2,2,1,1]]

## Univariate methods

In this section we cover two methods:
- CART,
- C4.5.

There are many more univariate methods. 

### CART

In [2]:
class BinaryLeaf:

    def __init__(self, elements, labels):
        self.L = None
        self.R = None
        self.elements = elements
        self.labels = labels
        self.completed = False

    def set_R(self, Rleaf):
        self.R = Rleaf

    def set_L(self, Lleaf):
        self.L = Lleaf

    def set_elements(self, elements):
        self.elements = elements

    def get_elements(self):
        return self.elements

    def set_p(self, threshold):
        self.p = threshold

    def get_L(self):
        return self.L

    def get_R(self):
        return self.R

    def set_completed(self):
        self.completed = True

    def is_completed(self):
        return self.completed

    def get_labels(self):
        return self.labels


In [3]:
class TreeUtils:

    def __init__(self, root):
        self.root_node = root

    def get_unique_labels(self, labels):
        return numpy.unique(numpy.array(labels)).tolist()

    def get_unique_values(self, elements):
        features_number = len(elements[0])
        unique = []
        for i in range(features_number):
            features_list = []
            for j in range(len(elements)):
                features_list.append(elements[j][i])
            unique.append(numpy.unique(numpy.array(features_list)))
        return unique

    def is_binary_leaf_completed(self, node):
        if node.is_completed():
            if node.get_L() != None and not node.get_L().is_completed():
                return node.get_L()
            elif node.get_R() != None and not node.get_R().is_completed():
                return node.get_R()
            elif node.get_L() == None and node.get_R() == None:
                return None
            elif node.get_L().is_completed() or node.get_R().is_completed():
                new_node = self.is_binary_leaf_completed(node.get_L())
                if new_node == None:
                    return self.is_binary_leaf_completed(node.get_R())
                else:
                    return new_node
            else:
                return None
        return node

    def find_binary_leaf_not_completed(self):
        return self.is_binary_leaf_completed(self.root_node)

    def is_leaf_completed(self, node):
        if node.is_completed():
            child_nodes = node.get_child_leafs()
            if len(child_nodes) == 0:
                return None
            is_child_to_return = False
            for i in range(len(child_nodes)):
                if not child_nodes[i].is_completed():
                    return child_nodes[i]
                else:
                    new_node = self.is_leaf_completed(child_nodes[i])
                    if new_node != None:
                        is_child_to_return=True
            if is_child_to_return:
                return new_node
        return node

    def find_leaf_not_completed(self):
        return self.is_leaf_completed(self.root_node)
    
    def print_tree(self, root_node):
        leafs_exists=True
        level = 0

        print("Level: " + str(level))
        print(str(root_node.get_elements()))
        print(str(root_node.get_labels()))

        current_nodes=[]
        childs = root_node.get_child_leafs()
        for i in range(len(childs)):
            current_nodes.append([root_node,childs[i]])

        while leafs_exists:
            level = level + 1
            print("Level: " + str(level))
            new_nodes = []
            for i in range(len(current_nodes)):
                print("ancestor: "+ str(current_nodes[i][0].get_elements()) + " " + str(current_nodes[i][0].get_labels()))
                print("child: "   + str(current_nodes[i][1].get_elements()) + " " + str(current_nodes[i][1].get_labels()))
                print("")
                if current_nodes[i][1].get_child_leafs() != None:
                    childs = current_nodes[i][1].get_child_leafs()
                    for j in range(len(childs)):
                        new_nodes.append([current_nodes[i][1],childs[j]])
            if len(new_nodes) == 0:
                leafs_exists = False
            else:
                current_nodes = new_nodes

    def print_binary_tree(self, root_node):
        leafs_exists=True
        level = 0

        print("Level: " + str(level))
        print(str(root_node.get_elements()))
        print(str(root_node.get_labels()))

        current_nodes=[]
        if root_node.get_L() != None:
            current_nodes.append([root_node, root_node.get_L()])
        if root_node.get_R() != None:
            current_nodes.append([root_node, root_node.get_R()])

        while leafs_exists:
            level = level + 1
            print("Level: " + str(level))
            new_nodes = []
            for i in range(len(current_nodes)):
                print("ancestor: "+ str(current_nodes[i][0].get_elements()) + " " + str(current_nodes[i][0].get_labels()))
                print("child: "   + str(current_nodes[i][1].get_elements()) + " " + str(current_nodes[i][1].get_labels()))
                print("")
                if current_nodes[i][1].get_L() != None:
                    new_nodes.append([current_nodes[i][1], current_nodes[i][1].get_L()])
                if current_nodes[i][1].get_R() != None:
                    new_nodes.append([current_nodes[i][1], current_nodes[i][1].get_R()])
            if len(new_nodes) == 0:
                leafs_exists = False
            else:
                current_nodes = new_nodes    

In [4]:
root = BinaryLeaf(X, labels)
utils = TreeUtils(root)
current_node = root
labels_count = 2

In [5]:
import copy

def get_split_candidates(unique_values):
    split_list = []
    for i in range(len(unique_values)):
        current_list=[]
        temp_list=copy.deepcopy(unique_values)
        current_list.append(temp_list[i])
        del temp_list[i]
        current_list.append(temp_list)
        split_list.append(current_list)
    return split_list

In [6]:
def get_number_of_labels_for_value(elements, column_id, label):
    count = 0
    if not isinstance(elements, list):
        elements_list = [elements]
    else:
        elements_list = elements

    column_elements = get_node_elements_column(column_id)

    for i in range(len(elements_list)):
        for j in range(len(column_elements)):
            if column_elements[j] == elements_list[i]:
                if current_node.labels[j] == label:
                    count = count + 1
    return count

In [7]:
def get_node_elements_column(column_id):
    return numpy.array(current_node.elements)[...,column_id].tolist()

In [8]:
def count_number_of_elements(elements, column_id):
    count = 0
    if isinstance(elements, list):
        column_elements = get_node_elements_column(column_id)
        for i in range(len(elements)):
            count = count + column_elements.count(elements[i])
    else:
        count = count + get_node_elements_column(column_id).count(elements)
    return count

In [9]:
import math

def calculate_phi(elements, column_id):
    global utils
    global current_node
    global labels_count
    t_l = count_number_of_elements(elements[0], column_id)
    t_r = count_number_of_elements(elements[1], column_id)
    p_l = t_l * 1.0 / len(current_node.elements) * 1.0
    p_r = t_r * 1.0 / len(current_node.elements) * 1.0

    sum_p = 0
    labels = utils.get_unique_labels(current_node.labels)
    for i in range(labels_count):
        p_class_t_l = (get_number_of_labels_for_value(elements[0], column_id, labels[i])*1.0)/(count_number_of_elements(elements[0], column_id)*1.0)
        p_class_t_r = (get_number_of_labels_for_value(elements[1], column_id, labels[i])*1.0)/(count_number_of_elements(elements[1], column_id)*1.0)
        sum_p = sum_p + math.fabs(p_class_t_l-p_class_t_r)
    return 2.0*p_l*p_r*sum_p

In [10]:
def check_completed(labels, elements):
    global utils
    ratio = len(utils.get_unique_labels(labels))
    if ratio == 1:
        return True
    elements=sorted(elements)
    duplicated = [elements[i] for i in range(len(elements)) if i == 0 or elements[i] != elements[i-1]]
    if len(duplicated) == 1:
        return True
    return False

In [11]:
def split_node(value, split_id):
    global current_node
    left_leaf = []
    left_leaf_labels = []
    right_leaf = []
    right_leaf_labels = []
    for i in range(len(current_node.elements)):
        if current_node.elements[i][split_id] == value:
            left_leaf.append(current_node.elements[i])
            left_leaf_labels.append(current_node.labels[i])
        else:
            right_leaf.append(current_node.elements[i])
            right_leaf_labels.append(current_node.labels[i])
    current_node.set_L(BinaryLeaf(left_leaf, left_leaf_labels))
    current_node.set_R(BinaryLeaf(right_leaf, right_leaf_labels))
    current_node.set_completed()
    if check_completed(left_leaf_labels, left_leaf):
        current_node.L.set_completed()
    if check_completed(right_leaf_labels, right_leaf):
        current_node.R.set_completed()

In [12]:
def get_current_node():
    global utils
    return utils.find_binary_leaf_not_completed()

In [15]:
import math

def calculate_omega(elements, column_id):
    global utils
    global current_node
    global labels_count
    t_l = count_number_of_elements(elements[0], column_id)
    t_r = count_number_of_elements(elements[1], column_id)
    p_l = t_l * 1.0 / len(current_node.elements) * 1.0
    p_r = t_r * 1.0 / len(current_node.elements) * 1.0

    sum_p = 0
    labels = utils.get_unique_labels(current_node.labels)
    for i in range(labels_count):
        p_class_t_l = (get_number_of_labels_for_value(elements[0], column_id, labels[i])*1.0)/(count_number_of_elements(elements[0], column_id)*1.0)
        p_class_t_r = (get_number_of_labels_for_value(elements[1], column_id, labels[i])*1.0)/(count_number_of_elements(elements[1], column_id)*1.0)
        sum_p = sum_p + math.fabs(p_class_t_l-p_class_t_r)
    return 2.0*p_l*p_r*sum_p

In [16]:
def build():
    global current_node
    global utils
    stop_criterion = False
    while stop_criterion == False:
        unique_values = utils.get_unique_values(current_node.get_elements())
        max_unique_id = 0
        max_split_id = 0
        max_value = None
        for i in range(len(unique_values)):
            if len(unique_values[i]) == 1:
                continue
            split_candidates = get_split_candidates(unique_values[i].tolist())
            for j in range(len(split_candidates)):
                current_value = calculate_omega(split_candidates[j],i)
                if max_value == None:
                    max_unique_id = i
                    max_split_id = j
                    max_value = current_value                    
                elif max_value < current_value:
                    max_unique_id = i
                    max_split_id = j
                    max_value = current_value
        split_node(unique_values[max_unique_id][max_split_id], max_unique_id)
        new_node = get_current_node()
        if new_node != None:
            current_node = new_node
        else:
            stop_criterion = True

In [17]:
build()

In [18]:
def show_tree():
    global root
    global utils
    utils.print_binary_tree(root)

In [19]:
show_tree()

Level: 0
[[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]]
[1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
Level: 1
ancestor: [[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
child: [[2, 1, 2, 2], [2, 3, 2, 2], [2, 2, 1, 2], [2, 3, 1, 2], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, -1, -1, 1, -1, -1, -1]

ancestor: [[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
child: [[1, 1, 2, 2], [1, 1, 1, 2], [1,

#### Exercise 1: Rewrite the CART method to use Gini index as shown in the lecture

Use the following equation:
\begin{equation}
I_{G}(X)=1-\sum_{i=1}^{m}p^{2}_{i},
\end{equation}
and
\begin{equation}
I_{G}(\text{feature})=1-\sum_{i=1}^{n}p_{i}*I_{G}(X_{i}),
\end{equation}



In [20]:
# your code goes here:




### C4.5

In [21]:
class Leaf:

    def __init__(self, elements, labels):
        self.child_leafs = []
        self.elements = elements
        self.labels = labels
        self.completed = False

    def get_elements(self):
        return self.elements

    def set_child_leafs(self, new_leafs):
        self.child_leafs = new_leafs

    def set_completed(self):
        self.completed = True

    def is_completed(self):
        return self.completed

    def get_labels(self):
        return self.labels

    def get_child_leafs(self):
        return self.child_leafs

In [22]:
root = Leaf(X, labels)
utils = TreeUtils(root)
current_node = root


In [23]:
def get_current_node():
    global utils
    return utils.find_leaf_not_completed()

In [24]:
def split(split_values, column_id):
    global current_node
    new_leafs = []
    for i in range(len(split_values)):
        indices = numpy.where(numpy.array(current_node.get_elements())[..., column_id].tolist() == split_values[i])
        new_leaf_elements = numpy.array(current_node.get_elements())[indices].tolist()
        new_leaf_labels   = numpy.array(current_node.get_labels())[indices].tolist()
        new_leaf = Leaf(new_leaf_elements,new_leaf_labels)
        if len(numpy.unique(new_leaf_labels)) == 1:
            new_leaf.set_completed()
        new_leafs.append(new_leaf)
    current_node.set_child_leafs(new_leafs)
    current_node.set_completed()

In [25]:
class Entropy:

    def calculate_entropy(self, labels):
        unique_labels, labels_count = numpy.unique(labels, return_counts=True)
        entropy = 0
        size = len(labels)
        for i in range(len(unique_labels)):
            if labels_count[i] > 0:
                log2=math.log((labels_count[i]*1.0)/(size*1.0),2)
            else:
                log2=0.0
            entropy=entropy-1.0*((labels_count[i]*1.0)/size)*log2
        return entropy

    def calculate_split_candidate_entropy(self, full_entropy, labels, elements, unique_labels, unique_elements, iter):
        split_entropy=0
        for i in range(len(unique_elements)):
            indices = numpy.where(numpy.array(elements)[..., iter].tolist() == unique_elements[i])
            unique_size = len(indices[0].tolist())
            filtered_labels = numpy.array(labels)[indices]
            for j in range(len(unique_labels)):
                labels_count = filtered_labels.tolist().count(unique_labels[j])
                if labels_count > 0:
                    log2=math.log((labels_count*1.0)/(unique_size*1.0),2)
                else:
                    log2=0.0
                split_entropy = split_entropy -1.0*((labels_count*1.0)/unique_size*1.0)*log2*unique_size*1.0/len(elements)*1.0
        return (full_entropy-split_entropy)


In [26]:
def build():
    global root
    global current_node
    stop_criterion = False
    current_node = root
    entropy = Entropy()
    unique_labels = utils.get_unique_labels(root.get_labels())
    while stop_criterion == False:
        unique_values = utils.get_unique_values(current_node.get_elements())
        full_entropy = entropy.calculate_entropy(current_node.get_labels())
        max_entropy_id = 0
        max_entropy_value = 0
        for i in range(len(unique_values)):
            split_entropy = entropy.calculate_split_candidate_entropy(full_entropy, current_node.get_labels(), current_node.get_elements(), unique_labels, unique_values[i], i)
            if split_entropy > max_entropy_value:
                max_entropy_id = i
                max_entropy_value = split_entropy
        split(unique_values[max_entropy_id], max_entropy_id)
        new_node = get_current_node()
        if new_node != None:
            current_node = new_node
        else:
            stop_criterion = True

In [27]:
build()

In [28]:
def show_tree():
    global utils
    global root
    utils.print_tree(root)

In [29]:
show_tree()

Level: 0
[[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]]
[1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
Level: 1
ancestor: [[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
child: [[1, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [1, 3, 2, 2], [1, 2, 1, 1]] [1, -1, 1, 1, 1]

ancestor: [[1, 1, 2, 2], [2, 1, 2, 2], [1, 1, 1, 2], [1, 2, 1, 2], [2, 3, 2, 2], [2, 2, 1, 2], [3, 2, 2, 1], [1, 3, 2, 2], [3, 3, 2, 1], [2, 3, 1, 2], [3, 1, 1, 1], [1, 2, 1, 1], [2, 3, 1, 1], [2, 1, 1, 2], [2, 2, 1, 1]] [1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1]
child: [[2, 1, 2, 2], [2, 3, 2, 2], [2, 2, 1, 2], [2, 3, 1, 2], [2, 3, 1, 1],

In [27]:
### Exercise 2: Use pydot do draw the tree for C4.5 example




## Multivariate decision trees

In this section we show how to develop the OC1 decision tree method.

OC1 classifier is divided into several steps:
1. Get all possible hyperplanes $H_{i}$.
2. Choose one.
3. Perturb and find $v_{j}$.
4. Calculate gini index of each $H_{i}$.
5. Choose $H_{i}$ with lowest gini index.

Let's import required libraries.

In [30]:
import numpy

We have implemented a TreeUtils class with some methods that we use later. Compared to the previous implementation is the new method **compare_two_leafs**.

In [31]:
class TreeUtils:

    def __init__(self, root):
        self.root_node = root

    def get_unique_labels(self, labels):
        return numpy.unique(numpy.array(labels)).tolist()

    def get_unique_values(self, elements):
        features_number = len(elements[0])
        unique = []
        for i in range(features_number):
            features_list = []
            for j in range(len(elements)):
                features_list.append(elements[j][i])
            unique.append(numpy.unique(numpy.array(features_list)))
        return unique

    def is_leaf_completed(self, node):
        if node.is_completed():
            if node.get_L() != None and not node.get_L().is_completed():
                return node.get_L()
            elif node.get_R() != None and not node.get_R().is_completed():
                return node.get_R()
            elif node.get_L() == None and node.get_R() == None:
                return None
            elif node.get_L().is_completed() or node.get_R().is_completed():
                new_node = self.is_leaf_completed(node.get_L())
                if new_node == None:
                    return self.is_leaf_completed(node.get_R())
                else:
                    return new_node
            else:
                return None
        return node

    def compare_two_leafs(self, leaf1, leaf2):
        labels1 = leaf1.labels
        labels2 = leaf2.labels
        if len(labels1) == len(labels2):
            for i in range(len(labels1)):
                if labels1[i] != labels2[i]:
                    return False
            return True
        return False

    def find_leaf_not_completed(self):
        return self.is_leaf_completed(self.root_node)

The Leaf class is the same as in C4.5 method.

In [32]:
class Leaf:

    def __init__(self, elements, labels):
        self.child_leafs = []
        self.elements = elements
        self.labels = labels
        self.completed = False

    def get_elements(self):
        return self.elements

    def set_child_leafs(self, new_leafs):
        self.child_leafs = new_leafs

    def set_completed(self):
        self.completed = True

    def is_completed(self):
        return self.completed

    def get_labels(self):
        return self.labels

    def get_child_leafs(self):
        return self.child_leafs

We use the same data set to make the comparison possible.

In [33]:
labels = [1,1,-1,1,-1,-1,1,1,1,1,1,1,-1,-1,-1]
X = [[1,1,2,2],[2,1,2,2],[1,1,1,2],[1,2,1,2],[2,3,2,2],
                [2,2,1,2],[3,2,2,1],[1,3,2,2],[3,3,2,1],[2,3,1,2],
                [3,1,1,1],[1,2,1,1],[2,3,1,1],[2,1,1,2],[2,2,1,1]]

Compare to C4.5 and CART we have one more variable R which is a parameter that is used to set the number of loops to randomly choose the feature to check if feature change can give better split. See build_level().

In [34]:
feature_number = len(X[0])
root = Leaf(X, labels)
utils = TreeUtils(root)
current_level = root
R = 10

The gini index can be calculated as shown below. Please keep in mind that it's only the gini index for a given split and need to be subtracted with 1 as shown in get_all_possible_splits_by_gini method.

In [35]:
def calculate_gini(labels):
    unique_labels = np.unique(labels)
    gini = 0
    for label in unique_labels:
        found = np.where(labels == label)
        gini = gini + len(found)/len(labels)
    return np.square(gini)

In the method below we calculated all possible hyperplane by calculating the gini indices for each feature. It is kind of similar to what we have done in CART method, but it will be "fixed" during the perturb part of the  OC1 method.

In [36]:
def get_all_possible_splits_by_gini(leaf):
    global feature_number
    data_set = leaf.elements
    labels = leaf.labels
    ginis = []
    for i in range(feature_number):
        feature_ginis = []
        feature_column = data_set[:, i]
        for feature in feature_column:
            distinguish = feature_column <= feature
            left_labels  = labels[distinguish]
            right_labels = labels[~distinguish]
            gini = 1 - calculate_gini(left_labels) + calculate_gini(right_labels)
            feature_ginis.append([feature,gini])
        ginis.append(min(feature_ginis))
    return ginis

We have also a method to find the current leaf to be splitted. It uses the utils that have implemented above.

In [37]:
def find_current_level_data():
    global utils
    return utils.find_leaf_not_completed()

In the method below we compute the $V_{j}$ which gives us the knowledge if a given object is above or below the hiperplane. It can be formulated as:
$\sum_{i=1}^{d}a_{i}x_{i}+a_{d+1}>0$, where $a_{1},\ldots,a_{d+1}$ are coefficients. In our case $a_{d+1}$ is our label.

In [38]:
def compute_v(element, scv):
    return np.sum(np.multiply(element, scv[:-1])) + scv[-1]

The next step is to divide objects in the leaf into two sets which are above and below the hyperplane.

In [39]:
def divide_data_hiperplane(leaf,scv):
    below = []
    above = []
    below_labels = []
    above_labels = []
    for i in range(len(leaf.elements)):
        v = self.compute_v(leaf.elements[i],scv) > 0
        if v:
            above.append(leaf.elements[i])
            above_labels.append(leaf.labels[i])
        else:
            below.append(leaf.elements[i])
            below_labels.append(leaf.labels[i])
    return np.array(below), np.array(above), np.array(below_labels), np.array(above_labels)

The coefficients that we have used above can be calculated as following:

In [40]:
def get_coefficiency(splits):
    scv = np.zeros(len(splits)+1)
    min_split_index = np.argmin(splits)
    scv[min_split_index] = 1
    scv[-1] = -splits[min_split_index][1]
    return scv

To compute the assignment array can be calculated as: $U_{j}=\frac{a_{m}x_{jm}-V_{j}}{x_{jm}}$.

In [41]:
def compute_u(element, scv, feature):
    return (scv[feature] * element[feature] - compute_v(element, scv)) / element[feature]

A short method for sorting the $U$ for the split can be implemented as below. We use it in the perturb function below.

In [42]:
def split(element):
    return np.sort(element)

Perturb function is the core part of the OC1 method. It calculates different gini indices for different feature combinations. We get the combination with best gini index. We "fix" the previously calculated coefficients.

In [43]:
def perturb(leaf, scv, feature, old_gini):
    u=[]
    for element in leaf.elements:
        u.append(compute_u(element, scv, feature))
    splits = split(np.array(u))
    am = []
    for split in splits:
        new_scv = scv
        new_scv[feature] = split
        below, above, below_label, above_label = divide_data_hiperplane(leaf, scv)
        gini = 1 - calculate_gini(below_label) + calculate_gini(above_label)
        am.append([new_scv, gini])
    am = np.array(am)
    best_split_index = np.argmin(am[:,1])
    if am[best_split_index][1] < old_gini:
        return am[best_split_index][1], am[best_split_index][0]
    elif am[best_split_index][1] == old_gini:
        if random() < 0.3:
            return am[best_split_index][1], am[best_split_index][0]
    return old_gini, scv

The build_level method combine the above functions and split the data into two leafs, assign it and/or stop building the tree if no more leafs to be divided are found.

In [44]:
def build_level():
    global R
    leaf = find_current_level_data()
    if leaf == None:
        return
    splits = get_all_possible_splits_by_gini(leaf)
    split_coefficiency_vector = get_coefficiency(splits)
    below,above, below_label, above_label = divide_data_hiperplane(leaf,split_coefficiency_vector)
    gini = 1 - calculate_gini(below_label) + calculate_gini(above_label)
    for c in range(R):
        feature = randint(0,len(leaf.elements[0])-1)
        gini, split_coefficiency_vector = perturb(leaf, split_coefficiency_vector, feature, gini)
        below, above, below_label, above_label = divide_data_hiperplane(leaf,split_coefficiency_vector)
    left_leaf = Leaf(below, below_label)
    right_leaf = Leaf(above, above_label)
    leaf.set_completed()
    if len(np.unique(below_label)) == 1:
        left_leaf.set_completed()
    if len(np.unique(above_label)) == 1:
        right_leaf.set_completed()
    if utils.compare_two_leafs(leaf, left_leaf) or utils.compare_two_leafs(leaf,right_leaf):
        leaf.set_completed()
    else:
        leaf.set_R(right_leaf)
        leaf.set_L(left_leaf)
    build_level()

Execute the level building function.

In [45]:
def build():
    build_level()

### Exercise 3:

Implement the reduced error pruning (REP) method and use it on the tree created with OC1 classifier. Calculate the error rate for each tree node and prune the tree whenever the error rate is lower compared to the child nodes.

[[1, 1, 2, 2],
 [2, 1, 2, 2],
 [1, 1, 1, 2],
 [1, 2, 1, 2],
 [2, 3, 2, 2],
 [2, 2, 1, 2],
 [3, 2, 2, 1],
 [1, 3, 2, 2],
 [3, 3, 2, 1],
 [2, 3, 1, 2],
 [3, 1, 1, 1],
 [1, 2, 1, 1],
 [2, 3, 1, 1],
 [2, 1, 1, 2],
 [2, 2, 1, 1]]

In [46]:
leafs_exists=True
root_node = root
current_nodes=[]
childs = root_node.get_child_leafs()
for i in range(len(childs)):
    current_nodes.append([root_node,childs[i]])

    while leafs_exists:
        new_nodes = []
        for i in range(len(current_nodes)):
            if current_nodes[i][1].get_child_leafs() != None:
                childs = current_nodes[i][1].get_child_leafs()
                for j in range(len(childs)):
                    new_nodes.append([current_nodes[i][1],childs[j]])
        if len(new_nodes) == 0:
            leafs_exists = False
        else:
            current_nodes = new_nodes

NameError: name 'current_nodes' is not defined